# Tutorial

The goal of this tutorial colab is to train the normalizing flow model proposed in reference [1] on a small system comprising 8 particles of monatomic water in the cubic ice phase. 

All the relevant code is available on github [2] as supplemental material to the publication but some parts of the logic are missing here and need to be implemented for the model to train.

Training the model with the hyperparameters (config) below does not require any hardware accelerators and the model should reach an ESS of about 10% on a public CPU kernel in under 10 minutes.


<br/>

**References**

[1] Wirnsberger, Papamakarios, Ibarz et al., *Normalizing flows for atomic solids*, Mach. Learn.: Sci. Technol. 3 025009 (2022), [link](https://iopscience.iop.org/article/10.1088/2632-2153/ac6b16).

[2] Supplemental code for *Normalizing flows for atomic solids* on github: [deepmind/flows_for_atomic_solids](https://github.com/deepmind/flows_for_atomic_solids).

[3] Jarzynski, *Targeted free energy perturbation*, Phys. Rev. E 65, 046122 (2002), [link](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.65.046122).

[4] Wirnsberger, Ballard et al., *Targeted free energy estimation via learned mappings*, J. Chem. Phys. 153, 144112 (2020), [link](https://doi.org/10.1063/5.0018903).

[5] Nicoli et al., *Asymptotically unbiased estimation of physical observables with neural samplers*, Phys. Rev. E 101, 023304 (2020), [link](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.101.023304).

## Imports

In [ ]:
!git clone https://github.com/deepmind/flows_for_atomic_solids.git
!pip install -r flows_for_atomic_solids/requirements.txt

In [ ]:
from typing import Callable, Dict, Tuple, Union
from absl import app
from absl import flags
import chex
import distrax
from flows_for_atomic_solids.experiments import monatomic_water_config
from flows_for_atomic_solids.experiments import utils
from flows_for_atomic_solids.utils import observable_utils as obs_utils
import haiku as hk
import jax
import jax.numpy as jnp
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib import rcParams
import optax

Array = chex.Array
Numeric = Union[Array, float]

rcParams.update({
    'font.size': 16, 'xtick.labelsize': 16, 'ytick.labelsize': 16,
    'legend.fontsize': 16, 'lines.linewidth': 3, 'axes.titlepad': 16,
    'axes.labelpad': 16, 'axes.labelsize': 20,
    'figure.figsize': [8.0, 6.0]})

## Model specification

We modify some hyperparameters in the default config from the paper to reduce the model size and to optimise for quick training on a CPU. With the settings below, the model should train to about 10% ESS in less than ten minutes on a CPU. The model will still improve if you train it for longer (by increasing `num_training_steps`).

In [ ]:
num_training_steps=501
config = monatomic_water_config.get_config(num_particles=8, lattice='cubic')
config.train.learning_rate=1e-3
config.model.kwargs.bijector.kwargs.num_layers=4
config.model.kwargs.bijector.kwargs.num_bins=16
config.model.kwargs.bijector.kwargs.conditioner.kwargs.num_frequencies=2
config.model.kwargs.bijector.kwargs.conditioner.kwargs.embedding_size=32
config.test.batch_size=16384
config.test.test_every=500
state = config.state

## Training the model

In this section, we define the training objective and a set of evaluation metrics that allows us to monitor the training progress. 

### Training objective

We train our model $q(x)$ to approximate the target Boltzmann distribution 

\begin{equation}
  p(x) = \frac{1}{Z} e^{-\beta U(x)}
\end{equation}


 by minimizing a Kullback&ndash;Leibler divergence as the loss function:

\begin{equation}
    D(q || p) = {\langle{\ln{ q(x)} - \ln{p(x)}}\rangle}_q = {\langle{\ln{ q(x)} + \beta U(x)}\rangle}_q + \ln{Z}.  
\end{equation}

The last term, $\ln Z$, is the logarithm of the normalizing constant and it can therefore be ignored as its gradient with respect to the model parameters vanishes. The function $U(x)$ is a given energy function (here the mW potential) and $\beta = (k_\text{B} T)^{-1}$ is the inverse temperature with $k_\text{B}$ being the Boltzmann constant.

In [ ]:
def get_loss(model: distrax.Distribution, energy_fn: Callable[[Array], Array],
             beta: Numeric, num_samples: int) -> Tuple[Array, Dict[str, Array]]:
  """Returns the loss and some additional metrics.

  Args:
    model: our model from which we can sample.
    energy_fn: a function that takes a batch of samples and returns a batch of
      energies.
    beta: the inverse temperature.
    num_samples: the number of samples to be used for computing the loss.

  Returns:
    The scalar loss and a dictionary containing energies, model log probs and 
    target log probs.

  Todo: 
    Draw a batch of samples from the model and implement the quantities that 
    are currently set to zero.
  """
  rng_key = hk.next_rng_key()

  zeros = jnp.zeros(config.train.batch_size)
  loss = 0.                # <ln q(x) + \beta U(x)>
  energy = zeros           # U(x)
  model_log_prob = zeros   # ln q(x)
  target_log_prob = zeros  # ln p(x)
  
  stats = {
      'energy': energy,
      'model_log_prob': model_log_prob,
      'target_log_prob': target_log_prob
  }
  return loss, stats

### Evaluation metrics

After every `config.test.test_every` training steps, we evaluate a set of metrics to monitor the training progress.
<br/><br/>

**Normalizing constant:**
To estimate $\ln Z$, we can use a targeted free energy estimator [3&ndash;5]. We first compute the forward work values

\begin{equation}
  \beta \Phi(x) = \beta U(x) + \ln{q(x)}
\end{equation}

and then the asymptotically unbiased estimate

\begin{equation}
\ln{Z} = \ln { \langle{\exp(-\beta \Phi(x)) \rangle }}_q.
\end{equation}
<br/>

**Expected energy:**
We can compute an unbiased estimate of the potential energy, ${\langle U \rangle}$, via importance sampling

\begin{equation}
{\langle U \rangle} = \frac{ \sum_n w_n U(x_n)} {\sum_n w_n},
\end{equation}

where $w_n = p^*(x_n)/q(x_n)$, $x_n \sim q(x)$ and $p^*(x) = Z p(x)$.
<br/><br/>

**Effective sample size (ESS):**
The effective samples size can be estimated as 

\begin{equation}
  {\text{ESS}} = \frac{ {\left( \sum_n w_n \right)}^2} {\sum_n w_n^2}.
\end{equation}
<br/>

**Helmholtz Free energy:**
Knowing the normalizing constant, we can compute the Helmholtz free energy $F$ via the relation

\begin{equation}
e^{-\beta F}  = \frac{ \ln Z}{N! \Lambda^{3N}},
\end{equation}

where $N$ is the number of particles in the system and
$\Lambda = 2.3925~\overset{\circ}{\text{A}}$ is the thermal de Broglie wavelength, which we set to the same value as the $\sigma$ parameter of the mW potential.



In [ ]:
def get_eval_metrics(loss: Array, energy: Array, model_log_prob: Array,
                     target_log_prob: Array) -> Dict[str, Array]:
  """Returns the evaluation metrics.

  Args:
    loss: a scalar containing the loss.
    energy: an array with shape [batch_size] containing energy values.
    model_log_prob: an array with shape [batch_size] containing log probs 
      under the model.
    target_log_prob: an array with shape [batch_size] containing log probs 
      under the target.

  Returns:
     A dictionary containing the evaluation metrics.

  Todo: 
    Implement the metrics that are currently set to zero.
  """
  energy_biased = 0.     # <U(x)>_q
  energy_unbiased = 0.   # <U(x)>_p
  ess = 0.               # ESS
  logz = 0.              # log Z
  beta_f = 0.            # \beta F/N
  metrics = {
      'loss': loss,
      'energy_biased': energy_biased,
      'energy_unbiased': energy_unbiased,
      'ess': ess,
      'logz': logz,
      'beta_f': beta_f,
  }
  return metrics

### Training loop


In [ ]:
def create_model():
  return config.model['constructor'](
      num_particles=state.num_particles,
      lower=state.lower,
      upper=state.upper,
      **config.model['kwargs'])

def train(num_iterations: int):
  energy_fn_train = config.train_energy.constructor(
      **config.train_energy.kwargs)
  energy_fn_test = config.test_energy.constructor(**config.test_energy.kwargs)
  lr_schedule_fn = utils.get_lr_schedule(
      config.train.learning_rate, config.train.learning_rate_decay_steps,
      config.train.learning_rate_decay_factor)
  optimizer = optax.chain(
      optax.scale_by_adam(),
      optax.scale_by_schedule(lr_schedule_fn),
      optax.scale(-1))
  if config.train.max_gradient_norm is not None:
    optimizer = optax.chain(
        optax.clip_by_global_norm(config.train.max_gradient_norm), optimizer)

  def loss_fn():
    """Loss function for training."""
    model = create_model()
    loss, stats = get_loss(
        model=model,
        energy_fn=energy_fn_train,
        beta=state.beta,
        num_samples=config.train.batch_size)
    train_metrics = dict(loss=loss, energy_biased=stats['energy'].mean())
    return loss, train_metrics

  def eval_fn():
    """Evaluation function."""
    model = create_model()
    loss, stats = get_loss(
        model=model,
        energy_fn=energy_fn_test,
        beta=state.beta,
        num_samples=config.test.batch_size)
    return get_eval_metrics(loss, **stats)
 
  print('Initialising system.')
  rng_key = jax.random.PRNGKey(config.train.seed)
  init_fn, apply_fn = hk.transform(loss_fn)
  _, apply_eval_fn = hk.transform(eval_fn)

  rng_key, init_key = jax.random.split(rng_key)
  params = init_fn(init_key)
  opt_state = optimizer.init(params)

  def _loss(params, rng):
    loss, metrics = apply_fn(params, rng)
    return loss, metrics

  jitted_loss = jax.jit(jax.value_and_grad(_loss, has_aux=True))
  jitted_eval = jax.jit(apply_eval_fn)
  
  step = 0
  print('Beginning of training.')
  while step < num_iterations:
    rng_key, loss_key = jax.random.split(rng_key)
    (_, metrics), g = jitted_loss(params, loss_key)

    if (step % 50) == 0:
      print(f'Train[{step}]: {metrics}')

    if (step % config.test.test_every) == 0:
      rng_key, val_key = jax.random.split(rng_key)
      metrics = jitted_eval(params, val_key)
      print(f'Valid[{step}]: {metrics}')

    # Update parameters.
    updates, opt_state = optimizer.update(g, opt_state, params)
    params = optax.apply_updates(params, updates)
    step += 1
    
  return params

params = train(num_training_steps)
print('Done')

Some reference values:
- $\langle U \rangle_p \approx -94.64~\text{kcal/mol}$
- $\beta F/N \approx -25.86$ 


## Analyzing the trained model

### Sampling from the model


In [ ]:
@hk.transform
def sample_and_log_prob(num_samples: int):
  """Returns samples and log probs from the base and the trained model."""
  key = hk.next_rng_key()
  model = create_model()
  return (
      model.sample_and_log_prob(seed=key, sample_shape=num_samples),
      model._base_model._flow_model.distribution.sample_and_log_prob(
          seed=key, sample_shape=num_samples))

((model_samples, model_log_probs), 
 (base_samples, base_log_probs)) = sample_and_log_prob.apply(params, jax.random.PRNGKey(42), 4096)

### Estimating normalised target log probs

In [ ]:
energy_fn = config.test_energy.constructor(**config.test_energy.kwargs)

def get_normalised_target_log_probs(  
    model_samples: Array,
    model_log_probs: Array) -> Array:
  """Returns the (approximately) normalised target log probs.

  Args:
    model_samples: samples drawn from the model.
    model_log_probs: model log probs for the `model_samples`.
 
  Returns:
    approximately normalised target log probs.

  Todo: 
    Estimate the normalising constant and use it to estimate the normalised
    target log probs: ln p(x) = -\beta U(x) - ln Z.
  """
  normalised_target_log_probs = jnp.zeros_like(model_log_probs)
  return normalised_target_log_probs

normalised_target_log_probs = get_normalised_target_log_probs(model_samples, model_log_probs)

### Plotting model vs. target log probs


In [ ]:
xmin = min(normalised_target_log_probs.min(), model_log_probs.min(), base_log_probs.min())
xmax = max(normalised_target_log_probs.max(), model_log_probs.max(), base_log_probs.max())
x = np.linspace(xmin, xmax, 100)
plt.xlim((xmin, xmax))
plt.ylim((xmin, xmax))
plt.scatter(normalised_target_log_probs, base_log_probs, c='blue', alpha=0.2, label='base')
plt.scatter(normalised_target_log_probs, model_log_probs, c='red', alpha=0.2, label='model')
plt.plot(x, x, linestyle='--', c='black')
plt.xlabel(r'$\ln \hat{p}(x)$')
plt.ylabel(r'$\ln q(x)$')
plt.legend()
plt.show()

### Plotting the radial distribution function

In [ ]:
box_length = config.test_energy.kwargs.box_length
model_rdf = obs_utils.radial_distribution_function(model_samples, box_length, num_bins=100)
base_rdf = obs_utils.radial_distribution_function(base_samples, box_length, num_bins=100)
plt.plot(base_rdf[:, 0], base_rdf[:, 1], linestyle='--', c='blue', label='base')
plt.plot(model_rdf[:, 0], model_rdf[:, 1], linestyle='-', c='red', label='model')
plt.xlabel(r'$r$')
plt.ylabel(r'$g(r)$')
plt.legend()
plt.show()